In [23]:
#import all dependencies
import cv2 as cv
import os
import numpy as np
import matplotlib.pyplot as plt
from imutils import perspective
from imutils import contours
import imutils

In [24]:
#reading video from file, counting it's frames
cap = cv.VideoCapture(os.path.join('D:','python_projects','testforwork','video_2023-06-03_19-34-39.mp4'))
frame_count = cap.get(cv.CAP_PROP_FRAME_COUNT)

#range of color for our 'greenscreen'
u_green = np.array([75, 255, 255])
l_green = np.array([38, 100, 100])

#reading image from file
img = cv.imread(os.path.join('D:','python_projects','testforwork','photo_2023-06-03_11-25-50.jpg'))

#getting dimensions of our image, storing it's coordinates
h1,w1 = img.shape[:2]
pts1=np.float32([[0,0],[w1,0],[0,h1],[w1,h1]])

kernel = np.ones((7,7),np.uint8)
rectangle_coords2= [[10, 5], [20, 5], [10, 8], [20, 8]]
rectangle_coords= [[10, 5], [20, 5], [10, 8], [20, 8]]
#rectangle_coords2= [[689, 463], [969, 463], [689, 649], [969, 649]]

In [25]:

fourcc = cv.VideoWriter_fourcc(*'XVID')
out = cv.VideoWriter('output.avi', fourcc, 20.0, (1600,  900))
#going through every frame
for frame_i in range(int(frame_count)):
    #getting frame from video, resizing window
    ret, frame = cap.read()
    small = (1600, 900)
    frame = cv.resize(frame, small, interpolation = cv.INTER_LINEAR)
    
    #removing green color from video, creating mask with inly green color
    hsv = cv.cvtColor(frame, cv.COLOR_BGR2HSV)
    mask = cv.inRange(hsv, l_green, u_green)
    mask = cv.morphologyEx(mask, cv.MORPH_CLOSE, kernel)
    res = cv.bitwise_and(frame,frame, mask= mask)
    f = frame - res
    
    #dimensions of video without green
    height, width = f.shape[:2]
    
    #finding edges of green rectangles
    gray = cv.cvtColor(res, cv.COLOR_BGR2GRAY)
    blur = cv.GaussianBlur(gray,(7, 7), 0)
    edges = cv.Canny(blur,100,200)
    edges = cv.dilate(edges, None, iterations=1)
    edges = cv.erode(edges, None, iterations=1)
    
    #drawing contour on edges
    contours = cv.findContours(edges.copy(), cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
    contours = contours[0]
    
    #going throug all contours
    for cnt in contours :
        #excluding small errors
        if cv.contourArea(cnt) < 500:
            continue
        
        #getting coordinates of corners of green rectangles, storing them in box list clockwise
        box = cv.minAreaRect(cnt)
        box = cv.cv.BoxPoints(box) if imutils.is_cv2() else cv.boxPoints(box)
        box = np.array(box, dtype="int")
        box = perspective.order_points(box)
        
        #we have two rectangles. Saving coordinates of first one in else, second one in if
        if np.max(box) > 1100 or np.max(box) > 1300:
            rectangle_coords2 = np.array([box[0], box[1], box[3], box[2]])
        else:
            rectangle_coords = np.array([box[0], box[1], box[3], box[2]])
    
    #storing coordinates as floats of rectangles
    pts2=np.float32(rectangle_coords)
    pts22=np.float32(rectangle_coords2)
    
    #finding homography for both rectangles
    h, masker = cv.findHomography(pts1, pts2, cv.RANSAC,5.0)
    h2, masker2 = cv.findHomography(pts1, pts22, cv.RANSAC,5.0)
    
    #changing image so it fits inside rectangles
    height, width, channels = f.shape
    im1Reg = cv.warpPerspective(img, h, (width, height))
    im2Reg = cv.warpPerspective(img, h2, (width, height))
    
    #putting everything together
    final = cv.bitwise_or(im1Reg, f)
    final2 = cv.bitwise_or(im2Reg, final)
    out.write(final2)
    #playing video
    cv.imshow('final.png',final2)
    k = cv.waitKey(5) & 0xFF
    if k == 27:
        break
cap.release()
cv.destroyAllWindows()

In [16]:
#cv.drawContours(orig, [box.astype("int")], -1, (0, 255, 0), 5)
        #for (xA, yA) in list(box):
        #    cv.circle(orig, (int(xA), int(yA)), 9, (0,0,255), -1)
        #    cv.putText(orig, "({},{})".format(xA, yA), (int(xA - 50), int(yA - 10) - 20), font, 0.5, (255,0,0), 2)